#### About

> Extractive text summarization.

In this method, We directly select few sentences from the input text which serve as a summary.

In [19]:
training_samples = [
    {
        'text': 'This is the first sentence. This is the second sentence. This is the third sentence.',
        'summary_indices': [0, 2]
    },
    {
        'text': 'This is another text. It has four sentences. The third sentence is important. The last sentence is not important.',
        'summary_indices': [0, 2]
    },
    {
        'text': 'This is a third text. It has three sentences. The second sentence is the most important one.',
        'summary_indices': [1]
    }
]


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

input_dim = 100 # the dimension of the input vector for each sentence
output_dim = 1 # the dimension of the output vector for each sentence

model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [23]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = np.zeros((len(training_samples), input_dim))
y = np.zeros((len(training_samples), output_dim))
tokenizer = Tokenizer(num_words=input_dim)

for i, sample in enumerate(training_samples):
    text = sample['text']
    summary_indices = sample['summary_indices']
    sentences = text.split('.')
    for j, sentence in enumerate(sentences):
        tokenizer.fit_on_texts([sentence])
        sequence = tokenizer.texts_to_sequences([sentence])
        sentence_vector = pad_sequences(sequence, maxlen=input_dim, padding='post')[0]
        X[i, :] = sentence_vector
        if j in summary_indices:
            y[i, :] = 1

model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6931 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6926 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6921 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6916 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6911 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6907 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6902 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6897 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6892 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6887 - accuracy: 1.0000


In [24]:
def generate_summary(model, input_text, input_dim):
    sentences = input_text.split('.')
    X = np.zeros((len(sentences), input_dim))
    for i, sentence in enumerate(sentences):
        sentence_vector = convert_sentence_to_vector(sentence, input_dim)
        X[i, :] = sentence_vector
    y_pred = model.predict(X)
    summary_indices = np.where(y_pred > 0.5)[0]
    summary = '.'.join([sentences[i] for i in summary_indices])
    return summary


In [25]:
def convert_sentence_to_vector(sentence, input_dim):
    tokenizer = Tokenizer(num_words=input_dim)
    tokenizer.fit_on_texts([sentence])
    sequence = tokenizer.texts_to_sequences([sentence])
    sentence_vector = pad_sequences(sequence, maxlen=input_dim, padding='post')[0]
    return sentence_vector

In [26]:
input_text = 'This is a long text with many sentences. Some sentences are more important than others. We want to generate a summary of the most important sentences.'
summary = generate_summary(model, input_text, input_dim)
print(summary)


1/1 [==============================] - 0s 108ms/step
 We want to generate a summary of the most important sentences.
